In [55]:
import numpy as np
import pandas as pd
# Importing of DataFrame with the 15 features (all except of 'truth')
from features_importance import x as dfNorm15, dfCorr
from newOWA_aggregation import v

# Defining of all singleton importance values mu

# mu1 length = dfNorm15.shape[1]+1 = 15 + 1 = number of features + mu(0)=0
mu1 = np.empty([1, dfNorm15.shape[1]+1])
mu1 = mu1.flatten()
for i in range(dfNorm15.shape[1]+1):
    if i == 0:
        mu1[i] = 0
    else:
        mu1[i] = v[i-1]/2

# Matrix of mu[i,j]

# mu2 shape = dfNorm15.shape[1] rows and dfNorm15.shape[1] cols = 15x15 = (0,14)x(0,14)
mu2 = np.zeros([dfNorm15.shape[1], dfNorm15.shape[1]])
#print(mu2)

# Calculation of mu[i, j] values
const_plus = 0.01
const_minus = 0.01
#const_plus = 0.10025
#const_minus = 0.3045

for i in range(dfNorm15.shape[1]):
    for j in range(dfNorm15.shape[1]):
        if i != j:
            # i/j + 1 only for adjusting indexes
            if (dfCorr.iloc[i,j] > 0):
                # corr POS: mu[i,j] > mu[i] + mu[j]
                mu2[i, j] = mu1[i+1] + mu1[j+1] + const_plus
            elif (dfCorr.iloc[i,j] < 0):
                # corr NEG: mu[i,j] < mu[i] + mu[j]
                mu2[i, j] = mu1[i+1] + mu1[j+1] - const_minus
        else:
            mu2[i, j] = None

# Assing NaN to the simmetric half of mu2
for i in range(mu2.shape[0]):
    for j in range(mu2.shape[1]):
        if j<i:
            mu2[i, j] = None

cols = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
df_mu2 = pd.DataFrame(mu2, columns=cols, index=cols)
#print(df_mu2)

In [56]:
for i in range(len(mu1)):
    print(i, mu1[i])

df_mu2

0 0.0
1 0.25
2 0.245
3 0.05
4 0.195
5 0.33
6 0.205
7 0.29
8 0.235
9 0.065
10 0.17
11 0.265
12 0.21
13 0.385
14 0.325
15 0.45


1      2      3      4      5      6      7      8      9      10     11  \
1  NaN  0.505  0.310  0.455  0.570  0.445  0.530  0.475  0.305  0.410  0.505   
2  NaN    NaN  0.285  0.450  0.565  0.440  0.525  0.470  0.300  0.425  0.500   
3  NaN    NaN    NaN  0.255  0.370  0.245  0.350  0.275  0.105  0.210  0.305   
4  NaN    NaN    NaN    NaN  0.515  0.410  0.495  0.420  0.250  0.355  0.450   
5  NaN    NaN    NaN    NaN    NaN  0.545  0.630  0.575  0.385  0.510  0.605   
6  NaN    NaN    NaN    NaN    NaN    NaN  0.505  0.450  0.260  0.365  0.480   
7  NaN    NaN    NaN    NaN    NaN    NaN    NaN  0.535  0.345  0.470  0.565   
8  NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  0.290  0.415  0.510   
9  NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  0.245  0.320   
10 NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  0.425   
11 NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
12 NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
13 NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
14 NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
15 NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

       12     13     14     15  
1   0.470  0.625  0.585  0.710  
2   0.465  0.620  0.560  0.685  
3   0.270  0.425  0.385  0.510  
4   0.415  0.590  0.530  0.655  
5   0.530  0.725  0.665  0.790  
6   0.405  0.600  0.540  0.665  
7   0.490  0.685  0.625  0.750  
8   0.435  0.630  0.570  0.695  
9   0.285  0.460  0.380  0.505  
10  0.390  0.565  0.485  0.610  
11  0.465  0.660  0.600  0.725  
12    NaN  0.585  0.545  0.670  
13    NaN    NaN  0.720  0.845  
14    NaN    NaN    NaN  0.785  
15    NaN    NaN    NaN    NaN

In [57]:
# Capacity constraints satisfaction

# MONOTONICITY constraint
# for each A,B € 2^N, [A C B -> mu1(A) <= mu2(B)]
def mono_capacity_constr(single_mu, pairs_mu):

    count_no = 0
    for i in range(pairs_mu.shape[0]):
        for j in range(pairs_mu.shape[1]):
            if not(np.isnan(pairs_mu[i,j])):
                if single_mu[i+1] <= pairs_mu[i, j] and single_mu[j+1] <= pairs_mu[i,j]:
                    #print('\nyes', i+1, '-', j+1, '\n', single_mu[i+1], 'and', single_mu[j+1], '<=', pairs_mu[i,j])
                    pass
                else:
                    count_no += 1
                    #print('\nNO!', i+1, '-', j+1, '\n', single_mu[i+1], 'and/or', single_mu[j+1], '>', pairs_mu[i,j])
    if count_no == 0:
        return('\nCapacity monotonicity SATISFIED!')
    else:
        return('\nCapacity monotonicity NOT satisfied!!!')


print(mono_capacity_constr(mu1, mu2))



# 2-additive fuzzy measure constraints satisfaction

# NORMALIZATION constraint
# SUM u[i,j] - (n-2) * SUM u[i]     n= 15
def normalization_constr(single_mu, pairs_mu, n_features):
    sumA, sumB = 0, 0

    for i in range(pairs_mu.shape[0]):
        for j in range(pairs_mu.shape[1]):
            if not(np.isnan(pairs_mu[i,j])):
                sumA += pairs_mu[i,j]


    for i in range(pairs_mu.shape[0]):
        sumB += single_mu[i]

    result = sumA - ((n_features - 2) * sumB)

    return result


# NON-NEGATIVITY constraint
# mu1[i] >=0 and < 1, mu2[i,j] > 0 and < 1
def non_negativity_constr(single_mu, pairs_mu):

    count_no = 0
    for i in range(pairs_mu.shape[0]):
        for j in range(pairs_mu.shape[1]):
            if not(np.isnan(pairs_mu[i,j])):
                if pairs_mu[i, j] <= 0 or pairs_mu[i, j] >= 1:
                    count_no += 1
                    print(pairs_mu[i, j])


    for i in range(pairs_mu.shape[0]):
        if single_mu[i] < 0 or single_mu[i] > 1:
            count_no += 1
            print(single_mu[i])


    if count_no == 0:
        return('\nNon-negativity SATISFIED!')
    else:
        return('\nNon-negativity NOT satisfied!!!')


# MONOTONICITY constraint
# SUM (mu2[i,k] - mu1[i]) >= (card(A) - 2) * mu1[k]
def monotonicity_constr(single_mu, pairs_mu):
    for i in range(pairs_mu.shape[0]):
        for j in range(pairs_mu.shape[1]):
            if i != j:
                # i/j + 1 only for adjusting indexes
                if (dfCorr.iloc[i, j] > 0):
                    # corr POS: mu[i,j] > mu[i] + mu[j]
                    pairs_mu[i, j] = single_mu[i + 1] + single_mu[j + 1] + const_plus
                elif (dfCorr.iloc[i, j] < 0):
                    # corr NEG: mu[i,j] < mu[i] + mu[j]
                    pairs_mu[i, j] = single_mu[i + 1] + single_mu[j + 1] - const_minus

    #print(pairs_mu)

    count_no = 0
    for i in range(pairs_mu.shape[0]):
        summ = 0
        for k in range(pairs_mu.shape[1]):
            if i != k:
                #print('(',pairs_mu[i,k],'-',single_mu[i+1],')')
                summ += (pairs_mu[i,k] - single_mu[i+1])
        #print(i+1, summ)
        if summ < (0 * single_mu[k+1]):
            count_no += 1


    if count_no == 0:
        return ('Monotonicity SATISFIED!')
    else:
        return ('Monotonicity NOT satisfied!!!')




print('\nNORMALIZATION constraint:', normalization_constr(mu1, mu2, dfNorm15.shape[1]))
print(non_negativity_constr(mu1, mu2))
print()
print(monotonicity_constr(mu1, mu2))


Capacity monotonicity SATISFIED!

NORMALIZATION constraint: 9.630000000000003

Non-negativity SATISFIED!

Monotonicity SATISFIED!
